# Download libraries if required

In [ ]:
#  %pip install monai==1.0.0 pytorch-ignite==0.4.11 autogen-agentchat~=0.2
# %pip install openai --upgrade

    extract-msg (<=0.29.*)
                 ~~~~~~~^
  Attempting uninstall: openai
    Found existing installation: openai 1.45.0
    Uninstalling openai-1.45.0:
      Successfully uninstalled openai-1.45.0
Note: you may need to restart the kernel to use updated packages.


# Import Libraries

In [1]:
from monai.bundle import run
from monai.bundle.config_parser import ConfigParser
import pandas as pd


/Users/puneet.jain/miniforge3/lib/python3.10/site-packages/ignite/handlers/checkpoint.py:17: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import ZeroRedundancyOptimizer


# Get classification for an Mammogram Image using Monai Model Zoo 

In [2]:
from typing import Annotated, Literal


def get_breast_density_classification(image_url : Annotated[str, "the local path of the image"]) -> dict:

    d= {"data": {
    "_target_": "createList.CreateImagefromJson",
    "J": {"image" : image_url}}}
    config_file = "../configs/inference_new.json"
    parser = ConfigParser()    
    parser.read_config(config_file)
    run(config_file=config_file, workflow_type="inference",**d)

    data = parser.get_parsed_content("output_dir")
    df = pd.read_csv(f"{data}/predictions.csv",header=None)
    return {"Output": chr(ord('@')+df[[1,2,3,4]].idxmax('columns')[0])}


In [3]:
image_path = "/Users/puneet.jain/Documents/Github/breast_density_classification/sample_data/D/sample_D3.jpg"
get_breast_density_classification(image_path)

2025-01-16 14:17:39,074 - INFO - --- input summary of monai.bundle.scripts.run ---
2025-01-16 14:17:39,075 - INFO - > config_file: '../configs/inference_new.json'
2025-01-16 14:17:39,076 - INFO - > workflow_type: 'inference'
2025-01-16 14:17:39,076 - INFO - > data: {'J': {'image': '/Users/puneet.jain/Documents/Github/breast_density_classification/sample_data/D/sample_D3.jpg'},
 '_target_': 'createList.CreateImagefromJson'}
2025-01-16 14:17:39,078 - INFO - ---




Loading dataset: 100%|██████████| 1/1 [00:00<00:00,  8.77it/s]
/Users/puneet.jain/miniforge3/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/puneet.jain/miniforge3/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


2025-01-16 14:17:39,481 - Engine run resuming from iteration 0, epoch 0 until 1 epochs
2025-01-16 14:17:39,620 - Restored all variables from ../models/model.pt


/Users/puneet.jain/miniforge3/lib/python3.10/site-packages/monai/handlers/checkpoint_loader.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(sel

2025-01-16 14:17:46,523 - Key metric: None best value: -1 at epoch: -1
2025-01-16 14:17:46,526 - Epoch[1] Complete. Time taken: 00:00:06.378
2025-01-16 14:17:46,526 - Engine run complete. Time taken: 00:00:07.044


/Users/puneet.jain/miniforge3/lib/python3.10/site-packages/monai/data/__init__.py:118: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  t = cls([], dtype=storage.dtype, device=storage.device)
/Users/puneet.jain/miniforge3/lib/python3.10/site-packages/monai/data/__init__.py:118: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  t = cls([], dtype=storage.dtype, device=storage.device)
/Users/puneet.jain/miniforge3/lib/python3.10/site-packages/monai/data/__init__.py:118: UserWarning: TypedStorage is deprecated. It will be r

{'Output': 'D'}

In [5]:
from dotenv import load_dotenv
from pathlib import Path
import os

dotenv_path = Path('.env')
load_dotenv(dotenv_path=dotenv_path)


True

In [6]:
import base64
from openai import OpenAI
from mlflow.utils.databricks_utils import get_databricks_host_creds

workspace_host = "https://adb-984752964297111.11.azuredatabricks.net"
endpoint = f"{workspace_host}/serving-endpoints"
token = os.environ['API_KEY']

# Initialize the OpenAI client with the Databricks serving endpoint
client = OpenAI(
    base_url=endpoint,
    api_key=token
)



# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Getting the base64 string
base64_image = encode_image(image_path)

response = client.chat.completions.create(
  model="intern-38b",
  messages=[
    {"role": "user", 
    "content": [
                {
                    "type": "image_url",
                    "image_url": {
                      "url": "https://www.arsenal.com/sites/default/files/styles/large_16x9/public/images/saka-celeb-bayern.png?h=3c8f2bed&auto=webp&itok=Twjeu8tug"
                    },
                },
            ],
    },
    # {"role": "user", 
    # "content": [
    #             {
    #                 "type": "text",
    #                 "text": "What's the weather like in San Francisco?"
            
    # }
    # ]
    # }
  ],
  tools = [{
    "type": "function",
    "function": {
        "name": "get_player_information",
        "description": "This functions gets all details about a player shown in the image including where he is male/female and club affiliation",
        "parameters": {
            "type": "object",
            "properties": {
                "team": {
                    "type":
                    "string",
                    "description":
                    "The sports club or team the player belongs to can be from any sport"
                },
                "gender": {
                    "type":
                    "string",
                    "description":
                    "The gender of the player can be either male or female or 'no player found'",
                },
            },
            "required": ["team", "gender"]
        }
    }
},
{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {"type": "string", "description": "City and state, e.g., 'San Francisco, CA'"},
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]}
            },
            "required": ["location", "unit"]
        }
    }
}
],
# tool_choice={"type": "function", "function": {"name": "get_player_information"}},
  temperature=0.1,
  max_tokens=200,

)

print(response)

2025-01-16 14:17:55,623 - HTTP Request: POST https://adb-984752964297111.11.azuredatabricks.net/serving-endpoints/chat/completions "HTTP/1.1 200 OK"
ChatCompletion(id='chatcmpl-378a9554a9c948709abc31a0c8cc062d', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='chatcmpl-tool-f1ea767e22b841ea9f81a1e63d1345ed', function=Function(arguments='{"team": "Arsenal", "gender": "male"}', name='get_player_information'), type='function')]), stop_reason=None)], created=1737037073, model='default', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=27, prompt_tokens=1109, total_tokens=1136, completion_tokens_details=None, prompt_tokens_details=None), prompt_logprobs=None)


In [7]:
import json
import os
import random
import time
from typing import Any, Callable, Dict, List, Optional, Tuple, Type, Union

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from termcolor import colored

import autogen
from autogen import Agent, AssistantAgent, ConversableAgent, UserProxyAgent
from autogen.agentchat.contrib.capabilities.vision_capability import VisionCapability
from autogen.agentchat.contrib.img_utils import get_pil_image, pil_to_data_uri
from autogen.code_utils import content_str

In [7]:
llm_config = {
    "config_list": [
        {
            "model": "intern-38b",
            "api_key": str(os.environ["API_KEY"]),
            "base_url": endpoint,
        }
    ],
    "temperature": 0.5,
    "max_tokens": 500
}

In [8]:
from typing import Annotated, Literal

Operator = Literal["+", "-", "*", "/"]


def calculator(a: int, b: int, operator: Annotated[Operator, "operator"]) -> dict:
    if operator == "+":
        return {"response" : int(a +b)}
    elif operator == "-":
        return {"response" : int(a - b)}
    elif operator == "*":
        return {"response" : int(a * b)}
    elif operator == "/":
        return {"response" : int(a / b)}
    else:
        raise ValueError("Invalid operator")

In [9]:
import os

from autogen import ConversableAgent

# Let's first define the assistant agent that suggests tool calls.
assistant = ConversableAgent(
    name="Assistant",
    system_message="You are a helpful AI assistant. "
    "You can help with simple calculations. "
    "Return 'TERMINATE' when the task is done.",
    llm_config={"config_list": llm_config['config_list']},
)

# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)

# Register the tool signature with the assistant agent.
assistant.register_for_llm(name="calculator", description="A simple calculator")(calculator)

# Register the tool function with the user proxy agent.
user_proxy.register_for_execution(name="calculator")(calculator)

2025-01-16 14:18:17,712 - Tried to attach usage logger `pyspark.databricks.pandas.usage_logger`, but an exception was raised: JVM wasn't initialised. Did you call it on executor side?


<function __main__.calculator(a: int, b: int, operator: Annotated[Literal['+', '-', '*', '/'], 'operator']) -> dict>

In [10]:
from autogen import register_function

# Register the calculator function to the two agents.
register_function(
    calculator,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    name="calculator",  # By default, the function name is used as the tool name.
    description="A simple calculator",  # A description of the tool.
)

/Users/puneet.jain/miniforge3/lib/python3.10/site-packages/autogen/agentchat/conversable_agent.py:2587: UserWarning: Function 'calculator' is being overridden.
  warnings.warn(f"Function '{tool_sig['function']['name']}' is being overridden.", UserWarning)
/Users/puneet.jain/miniforge3/lib/python3.10/site-packages/autogen/agentchat/conversable_agent.py:2506: UserWarning: Function 'calculator' is being overridden.
  warnings.warn(f"Function '{name}' is being overridden.", UserWarning)


In [11]:
chat_result = user_proxy.initiate_chat(assistant, message="What is (44232+654)*5 ?",silent = False)

User (to Assistant):

What is (44232+654)*5 ?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
2025-01-16 14:18:23,039 - HTTP Request: POST https://adb-984752964297111.11.azuredatabricks.net/serving-endpoints/chat/completions "HTTP/1.1 200 OK"
[autogen.oai.client: 01-16 14:18:23] {351} WARNING - Model default is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
2025-01-16 14:18:23,042 - Model default is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
Assistant (to User):

***** Suggested tool call (chatcmpl-tool-f8f507724d0941ff826f6db69e112b4d): calculator *****
Arguments: 
{"a": 44232, "b": 654, "operator": "+"}
********************************************************************************************

-----------------

In [14]:
import os

from autogen import ConversableAgent
from autogen.agentchat.contrib.multimodal_conversable_agent import MultimodalConversableAgent

# Let's first define the assistant agent that suggests tool calls.
assistant_p = MultimodalConversableAgent(
    name="image-explainer",
    max_consecutive_auto_reply=10,
    llm_config={"config_list": llm_config['config_list']},
    system_message= "you are an mammogram image analyser your job is to classify the breast density based on the mammogram provided use the tools in your you have specified and give explanation about the implications"
    "Return 'TERMINATE' when the task is done."
)

# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)

# Register the tool signature with the assistant agent.
assistant.register_for_llm(name="get_breast_density_classification", 
                           description="This a function which takes an input path which can be local path and gives classification of Density of the mammogram")(get_breast_density_classification)

# Register the tool function with the user proxy agent.
user_proxy.register_for_execution(name="get_breast_density_classification")(get_breast_density_classification)

/Users/puneet.jain/miniforge3/lib/python3.10/site-packages/autogen/agentchat/conversable_agent.py:2587: UserWarning: Function 'get_breast_density_classification' is being overridden.
  warnings.warn(f"Function '{tool_sig['function']['name']}' is being overridden.", UserWarning)


<function __main__.get_breast_density_classification(image_url: typing.Annotated[str, 'the local path of the image']) -> dict>

In [15]:
from autogen import register_function

# Register the calculator function to the two agents.
register_function(
    get_breast_density_classification,
    caller=assistant_p,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    name="get_breast_density_classification",  # By default, the function name is used as the tool name.
    description="This a function which takes an input path which can be local path and gives classification of Density of the mammogram",  # A description of the tool.
)

In [20]:
from typing import Annotated, Literal


def get_breast_density_classification(image_url : Annotated[str, "the local path of the image"]) -> dict:

    # d= {"data": {
    # "_target_": "createList.CreateImagefromJson",
    # "J": {"image" : image_url}}}
    # config_file = "../configs/inference_new.json"
    # parser = ConfigParser()    
    # parser.read_config(config_file)
    # run(config_file=config_file, workflow_type="inference",**d)

    # data = parser.get_parsed_content("output_dir")
    # df = pd.read_csv(f"{data}/predictions.csv",header=None)
    return {"Outputs" : "D"}

In [18]:
# Ask the question with an image
# open method used to open different extension image file
# Imports PIL module
from PIL import Image
im = Image.open(image_path)

user_proxy.initiate_chat(
    assistant_p,
    message = 
        {

                "content": 
                    [   {"type": "text","text" : f" local path of the mammogram  {image_path}"},
                        {"type": "image_url", "image_url": {"url": im}},
                        # {"type": "text","text": " classify the density of the breast?"}
                        ],
            }
)

User (to image-explainer):

 local path of the mammogram  /Users/puneet.jain/Documents/Github/breast_density_classification/sample_data/D/sample_D3.jpg<image>

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
2025-01-16 14:24:21,117 - HTTP Request: POST https://adb-984752964297111.11.azuredatabricks.net/serving-endpoints/chat/completions "HTTP/1.1 200 OK"
[autogen.oai.client: 01-16 14:24:21] {351} WARNING - Model default is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
2025-01-16 14:24:21,119 - Model default is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
image-explainer (to User):

***** Suggested tool call (chatcmpl-tool-e7eec9a0b8a440faab949970a77e6b5d): get_breast_density_classification *****
Arguments: 
{"image_u

Loading dataset: 100%|██████████| 1/1 [00:00<00:00, 10.47it/s]
/Users/puneet.jain/miniforge3/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/puneet.jain/miniforge3/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


2025-01-16 14:24:21,447 - Engine run resuming from iteration 0, epoch 0 until 1 epochs


/Users/puneet.jain/miniforge3/lib/python3.10/site-packages/monai/handlers/checkpoint_loader.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(sel

2025-01-16 14:24:21,490 - Restored all variables from ../models/model.pt


/Users/puneet.jain/miniforge3/lib/python3.10/site-packages/ignite/handlers/checkpoint.py:17: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import ZeroRedundancyOptimizer
/Users/puneet.jain/miniforge3/lib/python3.10/site-packages/ignite/handlers/checkpoint.py:17: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import ZeroRedundancyOptimizer
/Users/puneet.jain/miniforge3/lib/python3.10/site-packages/ignite/handlers/checkpoint.py:17: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import Zero

2025-01-16 14:24:27,719 - Key metric: None best value: -1 at epoch: -1
2025-01-16 14:24:27,721 - Epoch[1] Complete. Time taken: 00:00:06.150
2025-01-16 14:24:27,721 - Engine run complete. Time taken: 00:00:06.274


/Users/puneet.jain/miniforge3/lib/python3.10/site-packages/monai/data/__init__.py:118: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  t = cls([], dtype=storage.dtype, device=storage.device)
/Users/puneet.jain/miniforge3/lib/python3.10/site-packages/monai/data/__init__.py:118: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  t = cls([], dtype=storage.dtype, device=storage.device)
/Users/puneet.jain/miniforge3/lib/python3.10/site-packages/monai/data/__init__.py:118: UserWarning: TypedStorage is deprecated. It will be r

User (to image-explainer):

User (to image-explainer):

***** Response from calling tool (chatcmpl-tool-e7eec9a0b8a440faab949970a77e6b5d) *****
[{'type': 'text', 'text': '{"Output": "D"}'}]
***************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
2025-01-16 14:24:47,409 - HTTP Request: POST https://adb-984752964297111.11.azuredatabricks.net/serving-endpoints/chat/completions "HTTP/1.1 200 OK"
[autogen.oai.client: 01-16 14:24:47] {351} WARNING - Model default is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
2025-01-16 14:24:47,412 - Model default is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
image-explainer (to User):

The mammogram prov

ChatResult(chat_id=None, chat_history=[{'content': [{'type': 'text', 'text': ' local path of the mammogram  /Users/puneet.jain/Documents/Github/breast_density_classification/sample_data/D/sample_D3.jpg'}, {'type': 'image_url', 'image_url': {'url': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1890x2457 at 0x346C61D20>}}], 'role': 'assistant', 'name': 'User'}, {'tool_calls': [{'id': 'chatcmpl-tool-e7eec9a0b8a440faab949970a77e6b5d', 'function': {'arguments': '{"image_url": "/Users/puneet.jain/Documents/Github/breast_density_classification/sample_data/D/sample_D3.jpg"}', 'name': 'get_breast_density_classification'}, 'type': 'function'}], 'content': None, 'role': 'assistant'}, {'content': '{"Output": "D"}', 'tool_responses': [{'tool_call_id': 'chatcmpl-tool-e7eec9a0b8a440faab949970a77e6b5d', 'role': 'tool', 'content': '{"Output": "D"}'}], 'role': 'tool', 'name': 'User'}, {'content': 'The mammogram provided is classified as Density type "D."\n\nBreast density is categorized into fo